# The "their" of "their kingdom" In 4Q246, II.2

<img src="images/their_kingdom.png"></img>

In [4Q246 (Aramaic)](http://www.deadseascrolls.org.il/explore-the-archive/manuscript/4Q246-1), a controversial figure appears who is called ברה די אל ("son of God") and בר עליון ("son of the Most High"). Much of the material before this point (column 2 line 1) is damaged, which complicates the identification of the figure. But the material which follows the reference leaves an open question as to whether the son of God is presented as a positive or negative character.

The relevant text of the passage in question is:

> ברה די אל יתאמר ובר עליון יקרונה <br>
> כזיקיא חזותא כן מלכותהן תהוה <br>
> שנין ימלכון על ארעא וכלא ידשון <br>
> ...<br>
> עד יקום עם אל

<br>
> The Son of God he will be called, and son of the Most High they will name him. <br>
> Like the sparks/comets of the vision, thus will their kingdom be. <br>
> Years they will reign over the land, and everyone they will trample. <br>
> ... <br>
> Until [the] people of God arise. (4Q246, Col 2, lns. 1-3, 5) <br>

A major issue in interpreting the "son of God" figure is the identification of the 3MP "they" between lines 1-2. Specifically, the text says, "...and son of the Most High **they** will name him" and then progresses to say, "Like the comets of the vision, thus will **their** kingdom be." Since the 3MP suffix on "their kingdom" refers to a tumultuous kingdom that ends after the people of God arise ("until the people of God..."), the "son of God" must be so named by a wicked people, that is, if the 3MP in both clauses refer to the same referent. 

## Research Question:

Are there places where a suffix in a clause identifies in person, gender, and number with a subject suffix in an immediately preceding clause, but refers to a different referent?

If so: What factors create a separation between the two referents?


## Inventory of Parameters

Based on the problem described above, here is a set of parameters the code must find:

* Two contiguous clauses
    * If clause is defective or causus pendens, it must be considered together with its last portion.
    * Clause 1 contains a third person verb with PGN X
    * Clause 2 contains a noun with a third person pronominal suffix with PGN X
    * No other PGN indicators can occur between PGN reference 1 and PGN reference 2
        * Exception: exceptions will be made for nouns connected to a prepositional phrase, since it is the case in 4Q246 that a noun with identical PGN markers intervenes between the verb and the pronominal suffix, but it is a comparative phrase with כ. The exception is made in the interest of finding examples as close to 4Q246 as possible.
    
Export results to a spreadsheet for examination. Hits will need to be manually labled as Y or N on the question: is the same referent? Include a separate row in the spreadsheet for this.

Begin first with Biblical Aramaic. If data is sparse, or if it is examined and further data is preferable, expand the search to Biblical Hebrew.

## See the Data Generated by this Query: 

* [Aramaic](results/contiguous_suffixes_BA.csv)
* [Hebrew](results/contiguous_suffixes_BH_Quotation.csv)


## Results:

This query finds 5 cases (of 405) in Biblical Hebrew quotative clauses where an identical verbal subject is separated from an identical pronominal suffix in a subsequent clause due to the use of an impersonal subject by the verb. Such an impersonal subject in 4Q246 for the verb, "they will name him" seems likely given that the verb is parallel to a passive form and given the tendency in Aramaic for 3MP impersonal verbs to occur in parallel to a passive verb. This notebook leads to a number of additional questions which will be addressed in the follow up notebook, `4Q256_their_kingdom_2.ipynb`.

In [1]:
from datetime import datetime
print(f'Notebook last updated on {datetime.now().__str__()}')

Notebook last updated on 2017-05-20 22:12:14.349493


In [2]:
# import tools
import collections, csv

# import TF and data
from tf.fabric import Fabric

# initialize TF
TF = Fabric(modules='hebrew/etcbc4c', silent=True)

# load features
api = TF.load('''
              book chapter verse language
              prs prs_gn prs_nu prs_ps
              gn nu ps
              function pdp typ rela domain
              ''')

api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.01s B book                 from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.00s B chapter              from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.01s B verse                from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B language             from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.13s B prs                  from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B prs_gn               from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B prs_nu               from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B prs_ps               from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.10s B gn                   from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.11s B nu                   from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.14s B ps            

### PGN Matching Functions

First we need some functions to match person, gender, and number indicators between pronominal suffixes -> verbs or nouns -> verbs. The key function is `match_pgn()`, which is made only for third person matching. Afterward, we provide some testing examples to ensure the algorithm works.

Those functions were originally written in this notebook. They have now been moved into the module, `pgn.py`, stored in the folder `participant_functions`. We import them now.

In [4]:
# import the special functions for person, gender, and number matching
import participant_functions.pgn as pgn

Testing the PGN matcher function...

In [6]:
pronA, cmpA = ('unknown','m','sg'),('p3','m','sg')
pronB, cmpB = ('p3','f','sg'),('unknown','f','sg')
pronC, cmpC = ('p3','m','sg'),('unknown','f','sg')
pronD, cmpD = ('NA', 'm', 'sg'),('p3', 'm', 'sg')

print('test A:', pgn.match_pgn(pronA, cmpA))
print('test B:', pgn.match_pgn(pronB, cmpB))
print('test C:', pgn.match_pgn(pronC, cmpC))
print('test D:', pgn.match_pgn(pronD, cmpD))

test A: True
test B: True
test C: False
test D: True


### Contiguous Verb Suffix // Pronominal Suffix Function

Now we build a big function to find third person verbs in one clause, and the pronominal suffixes that agree with them in person, gender, and number. There can be no intervening PGN markers between these two points of reference that agree with the verb. Thus, there is, in theory, an unbroken chain of PGN indicators up to the suffix. This will allow for testing the problem as it appears in 4Q246. 

In [7]:
def get_contiguous_suffixes(clause_atoms):
    '''
    Return a list of dictionary matches that have:
        * Clause 1 with verbal suffix X
        * Clause 2 with noun + pronominal suffix X
        * No interrupting PGN markers between the suffixes
    Dictionary contains row data for a spreadsheet.
    Requires a list of clauses in canonical order.
    '''
    
    # print info to console with TF.info()
    indent(level=0, reset=True)
    info('Beginning search for contiguous suffixes...')
    indent(level=1, reset=True)
    
    # put results here
    matches = []
    
    # find matches
    for ca in clause_atoms:
        
        # |1| Check for third person predicate verb.
        
        # get third person verb
        verb = [word for word in L.d(ca, otype='word') # get all words in clause atom
                    if F.function.v(L.u(word, otype='phrase')[0]) == 'Pred' # predicate word 
                    and F.pdp.v(word) == 'verb' # word must be verb
                    and F.ps.v(word) == 'p3' # verb must be third person
               ]
        
        # skip if no verb or third person verb
        if not verb:
            continue
            
        # take the last verb (in case there are multiples)
        verb = verb[-1]
        
        # |2| Check subsequent clause atom for noun with pronominal suffix.
        
        # get subsequent clause atom
        subsequent_ca = ca + 1 if ca + 1 in clause_atoms else None
        
        # skip if there is no following clause atom
        if not subsequent_ca:
            continue
            
        # get all words in subsequent ca 
        subs_words = L.d(subsequent_ca, otype='word')
            
        # if the subsequent ca is causus or defective, check the resumptive clause too
        if F.typ.v(subsequent_ca) in {'CPen','Defc'}\
            and F.rela.v(subsequent_ca + 1) == 'Resu':
                # get resumption clause atom
                resume_ca = subsequent_ca + 1
                # extend subsequent words with the words in the resumption clause atom
                subs_words.extend(L.d(resume_ca, otype='word'))
        
        # get all words in subsequent words with 
        # a pronominal suffix that agree with the verb
        suffixed_nouns = [word for word in subs_words
                              # word functions as noun
                              if F.pdp.v(word) == 'subs'
                               # word has pronominal sfx
                              and F.prs.v(word) not in {'NA','absent'}
                              # PGN agree
                              and pgn.match_pgn(pgn.get_pgn(word, pronom=True), pgn.get_pgn(verb))
                         ] # end list
        
        # skip if there are no nouns with pronominal suffixes
        if not suffixed_nouns:
            continue
            
        # take first suffixed noun
        suff_noun = suffixed_nouns[0]
            
        # |3| Check every word in between the verb in clause 1 
        # and the suffixed noun in clause 2
        # for any PGN marker that agrees with the verb.
        # Skip ca if there is an interrupting marker.
        
        # get any intervening PGN markers
        intervening_words = [word for word in range(verb+1,suff_noun)
                                # not prep phrs
                                if F.typ.v(L.u(word, otype='phrase')[0]) != 'PP'
                                # agrees with verb
                                and pgn.match_pgn(pgn.get_pgn(word),pgn.get_pgn(verb)) 
                                # has sfx
                                or pgn.match_pgn(pgn.get_pgn(word, pronom=True), pgn.get_pgn(verb))
                            ]
        
        # skip if there is an intervening word
        if intervening_words:
            continue
        
        # |4| Any clause atom up to this point is a match!
        
        # save and append match data
        
        # format reference
        book, chapter, verse1, = T.sectionFromNode(ca)
        verse2 = T.sectionFromNode(subsequent_ca)[2]
        reference = f'{book} {chapter}:{verse1}' if verse1 == verse2\
                        else f'{book} {chapter}:{verse1}-{verse2}'
        
        # format rest of data
        match_data = {'reference':reference,
                      'text':T.text(L.d(ca, otype='word') + L.d(subsequent_ca, otype='word')),
                      'same?': '', # empty column for manually sorting
                      'notes':'', # empty column for notes on sorting
                      'verb':T.text([verb]),
                      'pronominal': T.text([suff_noun]),
                      'verb_PGN': pgn.get_pgn(verb),
                      'suffx_PGN': pgn.get_pgn(suff_noun, pronom=True),
                      'vb_Cl_type': F.typ.v(ca), # clause type of verb
                      'pn_Cl_type': F.typ.v(subsequent_ca), # cl type of pronom
                      'verb_ca': ca,
                      'pron_ca': subsequent_ca
                     }
        
        # append the match data
        matches.append(match_data)
        
        # print updates for larger datasets
        if len(matches) % 50 == 0:
            info(f'{len(matches)} matches found...')
        
    # give final report
    indent(level=0)
    info(f'DONE with {len(matches)} results.')
    
    # return the matches
    return matches

### Apply the Function to Aramaic Texts and Export the Results

In [8]:
# get all clauses in Aramaic

aramaic_clauses = [ca for ca in F.otype.s('clause_atom')
                      if F.language.v(L.d(ca, otype='word')[0]) == 'arc'
                  ]

print(f'{len(aramaic_clauses)} Aramaic clause atoms ready for processing...')

1378 Aramaic clause atoms ready for processing...


In [9]:
# apply the function to the clauses

aramaic_results = get_contiguous_suffixes(aramaic_clauses)

  0.00s Beginning search for contiguous suffixes...
  0.09s DONE with 8 results.


In [10]:
# export the results

# fieldnames for columns
fieldnames = ('reference','text','same?','notes','verb','pronominal',
              'verb_PGN','suffx_PGN', 'vb_Cl_type','pn_Cl_type','verb_ca',
              'pron_ca')

aramaic_file = 'results/contiguous_suffixes_BA.csv'

# export commented out to prevent overwrite of manually edited files
# with open(export, 'w') as outfile:
#    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    
#    writer.writeheader()
#    writer.writerows(aramaic_results)

### For Hebrew

There are too many Hebrew results without an additional filter. So we filter by Hebrew clauses that only appear within the `domain` of "Q" (quotation). 

In [11]:
hebrew_clauses = [ca for ca in F.otype.s('clause_atom')
                      if F.language.v(L.d(ca, otype='word')[0]) == 'hbo'
                      and F.domain.v(L.u(ca, otype='clause')[0]) == 'Q'
                  ]

print(f'{len(hebrew_clauses)} Hebrew clause atoms ready for processing...')

53238 Hebrew clause atoms ready for processing...


In [12]:
# apply the function to the clauses

hebrew_results = get_contiguous_suffixes(hebrew_clauses)


  0.00s Beginning search for contiguous suffixes...
   |     0.77s 50 matches found...
   |     1.97s 100 matches found...
   |     3.31s 150 matches found...
   |     4.17s 200 matches found...
   |     4.93s 250 matches found...
   |     6.36s 300 matches found...
   |     7.11s 350 matches found...
   |     8.37s 400 matches found...
  8.69s DONE with 405 results.


In [13]:
hebrew_file = 'results/contiguous_suffixes_BH_Quotation.csv'

# export commented out to prevent overwrite of manually edited files
#with open(Heb_export, 'w') as outfile:
#    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    
#    writer.writeheader()
#    writer.writerows(hebrew_results)
    
# print('Done!')

## See Results

* [Aramaic](results/contiguous_suffixes_BA.csv)
* [Hebrew](results/contiguous_suffixes_BH_Quotation.csv)

# Analysis of Manually Categorized Results

Now that the results above have been exported and manually tagged as to whether the verb and pronominal suffix refers to the same referent, we can now re-import the data, now with the manual tags (y for "yes" it is the same referent or n for "no" it is not the same referent).

The goal of the analysis is to answer the question: When the referent is different between a verb and pronominal suffix with the same PGN data, and without an interrupting noun of the same PGN, what kind of factors are present? In other words: what kinds of factors lead to the disconnect of the pronominal suffix from the adjacent, agreeing verb?

Finally, what do these results bear out for our understanding of the same problem in 4Q246?

Since there were only about 8 Aramaic results, we focus on the Hebrew first.

After the results were manually labeled, they were saved within the same directory by the same names. We will access their data by simply re-opening them.

In [14]:
# (re)load the manually labeled data
with open(hebrew_file, 'r') as hfile:
    reader = csv.DictReader(hfile)
    
    # put results in list
    hebrew_results = [r for r in reader]
    
print(f'{len(hebrew_results)} Hebrew results loaded...')

405 Hebrew results loaded...


## Simple Data Exploration

What kind of results did the search find?

In [15]:
# A counting object to count the kinds of results
referent_match_counts = collections.Counter()

# add the manual categorizations to the count
for result in hebrew_results:
    referent_match_counts[result['same?']] += 1
    
# print the results
for category, count in referent_match_counts.items():
    print(category, count)

y 342
n 57
? 6


Thus: in 342 cases, the referent of the two suffixes is identical. In 57 cases (higher than I expected!), the referent is different. In 6 cases I chose to forego a formal classification because of the initial difficulties in disambiguating the referents. Now let's dive deeper into the 'no' results.

There are tags on each 'no' result that gives a clue as to the reason the referents are different. Let's peak into those tags and their counts.

In [16]:
# counter object to count the reasons for "no"
no_notes = collections.Counter()

# count the no tags
for result in hebrew_results:
    
    # count if result is tagged as no
    if result['same?'] == 'n':
        
        # up the count
        no_notes[result['notes']] += 1
        
    
# print the results from most to least
for category, count in sorted(no_notes.items(), key=lambda k: k[-1], reverse=True):
    print(category)
    print('\t', count)
    print()

additional_noun
	 36

new_referent
	 12

placeholder_referent
	 5

marked_discourse
	 2

fronted_element
	 2



The meaning of the tags are as follows:

* **additional_noun** - there is an additional noun subject, often it is a fronted, preverbal subject (hence it does not interrupt between the verb and the matched pronominal suffix). Occasionally the two subjects are a clause or few away. The interpretation of the referent then depends on a choice between two explicitly stated subjects.
* **new_referent** - there is an intervening noun subject between the verb and the pronominal suffix that was allowed in the search results since I made an exception for prepositional phrases.
* **placeholder_referent** - the verb is in the passive and has no actual referent, yet it agrees in PGN with pronominal suffix, which actually has a real referent (since such an exception occurs in 4Q246). 
* **marked_discourse** - the verb and the pronominal suffix are separated by the beginning of marked discourse...the verb is thus a verb of speaking. The two referents are different.
* **fronted_element** - there are two examples in Numbers in which a fronted element sets off a new section and separates the verb from the pronominal suffix. 

## placeholder_referent

First, the results with the tag `placeholder_referent` are of particular interest to the 4Q246 problem. In the 4Q246 text, there are parallel statements, one in the passive, and one in the active 3MP, before the clause with the pronominal suffix:

> ברה די אל יתאמר <br>
> ובר עליון יקרונה <br>
> "The son of God **he will be called**,<br>
> and son of the Most High **they** will name him."

While some have argued that the itpeel in this verse is reflexive, demonstrating the boastful nature of the title, Tucker Ferda, a proponent of reading the "son of God" title as a positive one, reads it as a passive. This leads him, in turn, to interpret the 3MP יקרונה as an unidentified subject that is simply in parallel to the hitpeel:

> One initial reason for suspicion is the third-person masculine plural verb in ii 1 (יקרונה), which questions the notion that the previous three Ithpeel/Ithpaal forms are reflexive. It instead suggests the opposite: the figure “is named” by some unspecified subject/s throughout (discussed more below). - "Naming the Messiah: A Contribution to the 4Q246 'Son of God' Debate," *Dead Sea Discoveries* 21 (2014): 165 

This reading goes hand in hand with Ferda's interpretation of the subsequent clause, which contains the "**their** kingdom" reference, which Ferda interprets as a separate antecedent. Thus, we look more closely at the `placeholder_referent` results. These tags represent situations in which a placeholder referent, required grammatically or simply with an unspecified antecedent, requires a distinction to be made between the verb referent (the placeholder) and the pronominal suffix referent.

In [17]:
# count matching results here
result_num = 0

# print out the text and verses of the 'no' results with the placeholder tag
for result in hebrew_results: # enumerate to ID results
    
    # print matching results
    if result['same?'] == 'n' and result['notes'] == 'placeholder_referent':
        
        # up the results count
        result_num += 1
        
        # print the results
        print(result_num)
        print(result['reference'])
        print(result['text'])
        print()

1
1_Samuel 6:3
וְנֹודַ֣ע לָכֶ֔ם לָ֛מָּה לֹא־תָס֥וּר יָדֹ֖ו מִכֶּֽם׃ 

2
1_Samuel 23:23
וְהָיָה֙ אִם־יֶשְׁנֹ֣ו בָאָ֔רֶץ 

3
Isaiah 66:23
וְהָיָ֗ה מִֽדֵּי־חֹ֨דֶשׁ֙ בְּחָדְשֹׁ֔ו וּמִדֵּ֥י שַׁבָּ֖ת בְּשַׁבַּתֹּ֑ו יָבֹ֧וא כָל־בָּשָׂ֛ר 

4
Joel 2:2-3
וְאַֽחֲרָיו֙ לֹ֣א יֹוסֵ֔ף עַד־שְׁנֵ֖י דֹּ֥ור וָדֹֽור׃ לְפָנָיו֙ אָ֣כְלָה אֵ֔שׁ 

5
Ecclesiastes 8:16
אֲשֶׁ֥ר נַעֲשָׂ֖ה עַל־הָאָ֑רֶץ כִּ֣י גַ֤ם בַּיֹּום֙ וּבַלַּ֔יְלָה שֵׁנָ֕ה בְּעֵינָ֖יו אֵינֶ֥נּוּ רֹאֶֽה׃ 



### Reflection on the "placeholder" or Impersonal Subjects

In all of the examples above, the PGN data in the verb is a dummy referent (first bolded word is the dummy): 
1. 1 Samuel 6:3 - "And **it** will be known to you why **his** hand does not turn aside from you [pl]."
2. 1 Samuel 23:23 - "And **it** will be if **he** is in the land..."
3. Isaiah 66:23 - "And **it** will be as often as a month in **its** month, and as often as a year in **its** year all flesh will come..."
4. Joel 2:2-3 - "And after it, **it** will not be again unto the years of many generations [generation and generation], before **it** a fire has consumed." 
    * Here the two "its" are harder to disambiguate. But the meaning of "it will not be again" seems to indicate more "there will not be [a day like it] again" See especially, though, the passive verb in the parallel statement just before it. 
<br><br>
5. Ecclesiastes 8:16 - "which [**it**] is done upon the earth, for indeed in the day and in the night he has slept in **his** eyes, and there was no one watching." 
    * Probably in this last example, the אשר serves as the antecedent, and this therefore should be relabled as `additional_noun`.
    
These results, which demonstrate that a PGN chain can be interrupted by a placeholder subject, may show that the two PGN referents in 4Q246 between lines II.1 and II.2 are actually different, which in turn would require a re-evaluation of the antecedent for pronominal suffix on מלכותהן ("their kingdom").

Upon looking up the placeholder phenomena in the grammars, it is recognized that the placeholder can also occur in the plural, especially in Aramaic. As per Gesenius:

> The 3rd plur. also is sometimes used to express an indefinite subject, where the context does not admit of a human agent or at least not of several, e.g. Gn 34:27. In such a case the 3rd plur. comes to be equivalent to a passive, as very commonly in Aramaic (see Kautzsch's *Gramm. des Bibl. Aram.*, §90 I *c*) -GKC §144*g*

And Kautzsch:
> Nicht selten steht so die 3. Pluralis ohne jede Buzugnamhme auf das wirkliche Subject der Handlung lediglich, um den Vollzug derselben auszudrücken und somit als Aequivalent eines Passivum...So Dan. 2, 30...4,13...4,22. 29...;4, 23... 5,20...5,21...7,12. 26. Ezr. 6,5. Kautzsch, *Grammatik*, §90 I *c*

And Rosenthal:
> The 3. masc. pl. and the masc. pl. of the participle frequently express an impersonal subject and thus substitute for a passive constriction..."they (one) will drive you out from mankind...and will make you eat grass like oxen." = "you will be driven out, etc." D 4:22 Rosenthal, *A Grammar of Biblical Aramaic*, §181, pg. 56.

This creates further questions and problems for the interpretation of 4Q246. If there is indeed a separation between the two antecedents of, "they will call him" and "their kingdom," then who is the antecedent of "their kingdom"? In the preceding column, there are only two potential options that are extant. The first consists of the 3MP of יעבדון ("they will make"; itself parallel with וכלא ישמשון  "And all will serve"). The second possibility is the (possibly) two kings of Assyria and Egypt, indicated simply in the extant portion of the damaged line 6, מלך אתור ומצרין. But can this construction truly indicate two kings? That shall be the subject of another query.

## Reflection on the Role of Semantic Similarity

There is an additional tag on some of the "yes" results in which semantic similarity between the two clauses served to clarify the connection between the antecedents. This will prove important for the next notebook query. 

Let's look at those tags and their corresponding texts. I only recognized this label toward the middle of my tagging work.

In [18]:
# find and print results with the semantic_similarity tag
for result in hebrew_results:
    
    # must be 'yes' result with semantic_similarity
    if result['same?'] == 'y' and result['notes'] == 'semantic_similarity':
        
        # print the results
        print(result['reference'])
        print(result['text'])
        print()

Psalms 78:3-4
וַ֝אֲבֹותֵ֗ינוּ סִפְּרוּ־לָֽנוּ׃ לֹ֤א נְכַחֵ֨ד׀ מִבְּנֵיהֶ֗ם 

Job 8:10
יֹ֣אמְרוּ לָ֑ךְ וּ֝מִלִּבָּ֗ם יֹוצִ֥אוּ מִלִּֽים׃ 

Job 31:38
אִם־עָ֭לַי אַדְמָתִ֣י תִזְעָ֑ק וְ֝יַ֗חַד תְּלָמֶ֥יהָ יִבְכָּיֽוּן׃ 



* Psalms 78:3-4; the similarity between the subject "father" and the noun upon which the pronominal suffix is affixed, "sons" indicates that the antecedent of the verb ספרו and בניהם is the same. 
* Job 8:10; the similarity between the verb יאמרו and מלים.
* Job 31:38; the similarity between תזעק and יבכיון. 

Upon reflection, these are not ideal representatives. Yet the question remains, does semantic similarities between one clause and another condition or limit the identification of the antecedent of a pronominal suffix? 

If, for instance, the 3MP of מלכותהן ("their kingdom") refers back to a מלך אתור ומצרין ("king of Assyria and Egypt"), the connection may occur due to the semantic similarity. The question remains to be investigated: are such semantic links common? To frame the problem into one which can be queried:

To affix a suffix to the noun "kingdom" suggests that the antecedent serves an active role over that kingdom. Identifiers, then, such as "king" could serve to connect the antecedent to the suffix reference. Is this true in Biblical Hebrew/Aramaic? What other kinds of nouns might be queried in this way?

# Final Reflections

This notebook has raised doubt about whether the 3MP of "their kingdom" in II.2 of 4Q246 truly refers back to the "they" of "they will name him" in II.1. But this possibility raises further important questions. 

1.&nbsp;&nbsp;If the two antecedents are different, then who would be the antecedent of "their kingdom"? There seem to be two main possibilities: the 3mp of יעבדון (I.8, though technically a damaged reading; כלא probably mirrors a no longer extant noun); OR, per Fitzmeyer, "The Aramaic 'Son of God'," 2000, 44, the compound noun phrase מלך אתור ומצרין actually refers to a plural entity, preceded by a plural (now lost) verb. Fitzmeyer reconstructs יאבדון ("they will perish"). Two 
issues need to be fleshed out before a decision between these two choices can be made:

   * Double check to make sure the plural construction as proposed by Fitzmeyer is acceptable. If not, then the "two kings" cannot be considered "two" but only "one". I know that compound subjects with plural verbs happens. How frequent is it? And what factors facilitate their identification?
   * Do semantic factors play a role in the identification of suffix antecedents? In the case of the word "kingdom" (מלכות, ממלכה) with pronominal suffixes, is the antecedent frequently identified as a מלך in the preceding narrative text? That would only seem natural. But how often does it happen? 
   * As corroborating evidence, are other prefixed nouns with connections to their antecedents indicated more explicitly by similar semantic content?
        
Both of these issues are deserving of their own queries and notebooks, which are to follow.

2.&nbsp;&nbsp;The disconnect between the "their" of "their kingdom" and the "they" of "they will name him" works in favor of reading the title as a positive one. But there remain difficulties for that interpretation. For instance, those who read the title as a positive, messianic one typically read the 3MS of מלכותה ("his kingdom") in II.5 as a reference to the ברה די אל ("the son of God"); yet this is in spite of the fact that the closest 3MS noun is not the בר but the עם אל ("people of God"). Proponents of the negative reading insist that the scribal vacat in II.4, " [vacat] Until the people of God arise..." creates a separate section from the preceding material, meaning that the עם אל is not only the closest option, but the only option. Yet, the grammatical structure itself suggests that the situation is more complicated than that. Importantly, the particle עד is not known elsewhere to indicate a clean break. Rather, it is used in dependent clauses. In this case, the עד clause quite naturally should connect back to the שנין ("years") in II.2, "years they will reign" (שנין ימלכון)..."until the people of God arise" (עד יקום עם אל). This suggests grammatical continuity; and it also creates an embedding of the tumultuous kingdom. This again raises the possibility, since there is not a complete break, that ברה די אל ("the son of God") could, conceivably, serve as the antecedent of the 3MS suffix on מלכותה. This depends on:
   * How does עד or similar temporal particles with a dependent relationship affect the inheritance of PGN information? Are there frequently identical indicators that are, in fact, separated due to the grammatical construction?

Likewise, this issue deserves a separate notebook.